# Sunco Oil Blending Problem

This is a classic Operations Research teaching problem. Taken from Operations Research: Applications and Algorithms by Wayne L. Winston, a cheat sheet can be found @ http://www.cpp.edu/~sparisay/Courses/SharedExamples/LP-blending/SuncoBlending-problem.doc

with a solution to the problem @ 
https://www.usna.edu/Users/math/dphillip/sa305.s13/sunco-sec2001.pdf

In this notebook we will use the package PuLP, a python package for linear programming to solve this LP.

## Import Libraries

In [1]:
from pulp import *
import cplex

## Define problem and decision variables

xij is the barrels of crude oil i used daily to produce gas j

aij is the dollars spend on daily advertising of gas

i = 1,2,3 and j =1,2,3

In [2]:
prob = LpProblem("Sunco Oil Blending", LpMaximize)

Hint: you can declare variables by looping through and appending to a list. However, we don't have a lot so let's declare them outright.

In [3]:
#Decision variable declaration
x11= LpVariable("Oil1_Gas1", lowBound=0,cat="continuous")
x12= LpVariable("Oil1_Gas2", lowBound=0,cat="continuous")
x13= LpVariable("Oil1_Gas3", lowBound=0,cat="continuous")
x21= LpVariable("Oil2_Gas1", lowBound=0,cat="continuous")
x22= LpVariable("Oil2_Gas2", lowBound=0,cat="continuous")
x23= LpVariable("Oil2_Gas3", lowBound=0,cat="continuous")
x31= LpVariable("Oil3_Gas1", lowBound=0,cat="continuous")
x32= LpVariable("Oil3_Gas2", lowBound=0,cat="continuous")
x33= LpVariable("Oil3_Gas3", lowBound=0,cat="continuous")
a1=LpVariable("Gas1_ad",lowBound=0,cat="continuous")
a2=LpVariable("Gas2_ad",lowBound=0,cat="continuous")
a3=LpVariable("Gas3_ad",lowBound=0,cat="continuous")

Staging data in dictionaries, but we can also use other methods such as dataframes.

In [4]:
prod_cost=4
ad_cost=10
gas1 = {
    "NAME": "Gas1",
    "SALES_PRICE":70.0,
    "MIN_OCATANE_RATING":10.0,
    "MAX_SULPHUR":0.01,
    "DEMAND":3000.0
}

gas2= {
    "NAME": "Gas2",
    "SALES_PRICE":60.0,
    "MIN_OCATANE_RATING":8.0,
    "MAX_SULPHUR":0.02,
    "DEMAND":2000.0,
}

gas3= {
    "NAME": "Gas3",
    "SALES_PRICE":50.0,
    "MIN_OCATANE_RATING":6.0,
    "MAX_SULPHUR":0.01,
    "DEMAND":1000.0
}

crude1 = {
    "NAME": "Crude1",
    "COST":45.0,
    "CAPACITY":5000.0,
    "OCTANE":12.0,
    "SULPHUR":0.5/100
}

crude2 = {
    "NAME": "Crude1",
    "COST":35.0,
    "CAPACITY":5000.0,
    "OCTANE":6.0,
    "SULPHUR":2.0/100
}

crude3 = {
    "NAME": "Crude1",
    "COST":25.0,
    "CAPACITY":5000.0,
    "OCTANE":8.0,
    "SULPHUR":3.0/100
}

## Define the objective function

The first "prob+=" should be the objective function. Pulp sees that there is no equation, but a statement and will identify this as the objective.

In [5]:
#Objective function - maximize daily profit
prob+=(gas1["SALES_PRICE"]-crude1["COST"]-prod_cost)*x11 + (gas2["SALES_PRICE"]-crude1["COST"]-prod_cost)*x12 + \
(gas3["SALES_PRICE"]-crude1["COST"]-prod_cost)*x13 +\
(gas1["SALES_PRICE"]-crude2["COST"]-prod_cost)*x21 + (gas2["SALES_PRICE"]-crude2["COST"]-prod_cost)*x22 + \
(gas3["SALES_PRICE"]-crude2["COST"]-prod_cost)*x23 +\
(gas1["SALES_PRICE"]-crude3["COST"]-prod_cost)*x31 + (gas2["SALES_PRICE"]-crude3["COST"]-prod_cost)*x32 + \
(gas3["SALES_PRICE"]-crude3["COST"]-prod_cost)*x33-a1-a2-a3


## Define the constraints

We use "prob+=" in the same way as the objective, but we have equalities or inequalities which pulp uses to identify these as constraints. You can also be creative with looping through a dataframe and variables and create many constraints at a time.

In [6]:
#Constrait 1 Gas 1 produced daily should equal its daily demand
prob += x11 + x21 + x31 - 10*a1 == gas1["DEMAND"]

#Constrait 2 Gas 2 produced daily should equal its daily demand
prob += x12 + x22 + x32 - 10*a2 == gas2["DEMAND"]

#Constrait 3 Gas 3 produced daily should equal its daily demand
prob += x13 + x23 + x33 - 10*a3 == gas3["DEMAND"]

# #constraints 4-6 5000 barrels of each crude can be purchased each day - but we can write this an easier way!
prob += x11 + x12 + x13 <= crude1["CAPACITY"]

prob += x21 + x22 + x23 <= crude2["CAPACITY"]

prob += x31 + x32 + x33 <= crude3["CAPACITY"]

# #max refinery capacity of 14000 barrels
prob += x11+x12+x13+x21+x22+x23+x31+x32+x33<=14000

# # #constraints 8-10 impose minimum octane levels on the LP

prob+= (crude1["OCTANE"]-gas1["MIN_OCATANE_RATING"])*x11+(crude2["OCTANE"]-gas1["MIN_OCATANE_RATING"])*x21+\
(crude3["OCTANE"]-gas1["MIN_OCATANE_RATING"])*x31 >=0.0

prob+= (crude1["OCTANE"]-gas2["MIN_OCATANE_RATING"] )*x12+(crude2["OCTANE"]-gas2["MIN_OCATANE_RATING"] )*x22+\
(crude3["OCTANE"]-gas2["MIN_OCATANE_RATING"] )*x32 >= 0.0

prob+= (crude1["OCTANE"]-gas3["MIN_OCATANE_RATING"])*x13+(crude2["OCTANE"]-gas3["MIN_OCATANE_RATING"])*x23+\
(crude3["OCTANE"]-gas3["MIN_OCATANE_RATING"])*x33 >= 0.0


# #constraint to impose max sulphur
prob+=(crude1["SULPHUR"]-gas1["MAX_SULPHUR"])*x11+(crude2["SULPHUR"]-gas1["MAX_SULPHUR"])*x21+\
(crude3["SULPHUR"]-gas1["MAX_SULPHUR"])*x31 <= 0.0
prob+=(crude1["SULPHUR"]-gas2["MAX_SULPHUR"])*x12+(crude2["SULPHUR"]-gas2["MAX_SULPHUR"])*x22+\
(crude3["SULPHUR"]-gas2["MAX_SULPHUR"])*x32 <= 0.0
prob+=(crude1["SULPHUR"]-gas3["MAX_SULPHUR"])*x13+(crude2["SULPHUR"]-gas3["MAX_SULPHUR"])*x23+\
(crude3["SULPHUR"]-gas3["MAX_SULPHUR"])*x33 <= 0.0

    

    

## You can write the LP to a text file (.lp can be opened with an editor)

On OSX, I use textedit to open this file. Windows can use notepad and linux can use gedit. Writing the LP to a file lets you see the LP symbolically, as if you were reading it mathematically.

In [7]:
prob.writeLP("sunco.lp")

## Solve the LP

In [8]:
prob.solve()
print ("Solution time: " + str(prob.solutionTime))

Solution time: 0.009210000000000051


In [9]:
print ("Status:", LpStatus[prob.status])

Status: Optimal


In [10]:
for v in prob.variables():
    print (v.name, "=", v.varValue)

Gas1_ad = 0.0
Gas2_ad = 750.0
Gas3_ad = 0.0
Oil1_Gas1 = 2222.2222
Oil1_Gas2 = 2111.1111
Oil1_Gas3 = 666.66667
Oil2_Gas1 = 444.44444
Oil2_Gas2 = 4222.2222
Oil2_Gas3 = 333.33333
Oil3_Gas1 = 333.33333
Oil3_Gas2 = 3166.6667
Oil3_Gas3 = 0.0


In [11]:
prob.objective.value()

287749.99967

In [12]:
%time 
prob.solve(CPLEX_PY(msg=0))
print ("Solution time: " + str(prob.solutionTime))

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
Solution time: 0.031021000000000076


In [13]:
#as the problem gets bigger, commercial solvers such as CPLEX and GUROBI will outperform CBC.

In [14]:
for v in prob.variables():
    print (v.name, "=", v.varValue)

Gas1_ad = 0.0
Gas2_ad = 750.0000000000002
Gas3_ad = 0.0
Oil1_Gas1 = 2088.8888888888887
Oil1_Gas2 = 2111.1111111111113
Oil1_Gas3 = 800.0
Oil2_Gas1 = 777.7777777777777
Oil2_Gas2 = 4222.222222222223
Oil2_Gas3 = 0.0
Oil3_Gas1 = 133.33333333333337
Oil3_Gas2 = 3166.6666666666674
Oil3_Gas3 = 200.00000000000006


In [15]:
prob.objective.value()

287750.0

You will notice that there are multiple optimal solutions! CPLEX tends to do a better job with decimal places and keeping the answer as an integer.